# Heat transfer 1D

This solve heat transfer by general formula.

The heat transfer formula is $c_1d^2T/dx^2+c_2T+c_3$

## Custom data
![](https://raw.githubusercontent.com/linnil1/2019FEM/master/images/Heat_1D.png)

* `heats`: we enter the element by tuple, in each tuple, it contains k(conduction), h(convection), l(length)
* `heat_p`: the Perimeter of each element(const)
* `heat_a`: the cross section area of each element(const)
* `lastF`: The environment temperture.
* `getC`, return the $c_1$, $c_2$, $c_3$
* `setBoundary`: set what you want. `bigT` stands for the matrix, `bigF` stands for external heat.

You should set your boundary. The default formula is for heat transfer. 

In [1]:
import numpy as np

# heat parameters of each elements
# k h l
heats = [(168, 30, 20e-3)] * 4
heat_p = 12e-3
heat_a = 5e-6
lastF = 20


# return c1 c2 c3 and L
def getC(heat):
    return heat[0] * heat_a, -heat[1] * heat_p, heat[1] * heat_p * lastF, heat[2]

# set boundary for the situation
def setBoundary():
    bigT[0, :] = 0
    bigT[0, 0] = 1
    bigF[0] = 100
    bigT[-1, -1] += heats[-1][1] * heat_a
    bigF[-1] += heats[-1][1] * heat_a * lastF

## solve

In [2]:
# parameters
n = len(heats) + 1

# set up basic stiffness matrix
bigT = np.zeros([n, n])
bigF = np.zeros(n)
for i in range(n - 1):
    c1, c2, c3, L = getC(heats[i])
    bigT[i:i+2, i:i+2] += c1 / L * np.array([[1, -1], [-1, 1]])
    bigT[i:i+2, i:i+2] -= c2 * L / 6 * np.array([[2, 1], [1, 2]])
    bigF[i:i+2] += c3 * L / 2 * np.array([1, 1])

# boundry
setBoundary()

# solve
print("bigT")
print(bigT)
print("bigF")
print(bigF)
t = np.linalg.inv(bigT).dot(bigF)
print("Result")
print(t)

bigT
[[ 1.       0.       0.       0.       0.     ]
 [-0.0408   0.0888  -0.0408   0.       0.     ]
 [ 0.      -0.0408   0.0888  -0.0408   0.     ]
 [ 0.       0.      -0.0408   0.0888  -0.0408 ]
 [ 0.       0.       0.      -0.0408   0.04455]]
bigF
[1.00e+02 1.44e-01 1.44e-01 1.44e-01 7.50e-02]
Result
[100.          75.03868587  59.79008101  51.56325515  48.90641549]


In [3]:
# post processing
print("loss")
heats = np.array(heats)
heat_loss = ((t[1:] + t[:-1]) / 2 - lastF) * heat_p * heats[:, 1] * heats[:, 2]
heat_loss = [*heat_loss, (t[-1] - lastF) * heats[-1, 1] * heat_a]
print(heat_loss)
print(np.sum(heat_loss))

loss
[0.4861392691230402, 0.34138356074377463, 0.25687201014282224, 0.21769081427295603, 0.0043359623229528455]
1.306421616605546
